In [7]:
import pandas as pd
import numpy as np
import shap
import joblib
import os

# Charger le dataset de 5000 clients
DATA_URL = "https://huggingface.co/datasets/Antonine93/projet7scoring/resolve/main/train_subset_with_target.parquet"
df = pd.read_parquet(DATA_URL)

# Sauvegarde des IDs clients pour les relier plus tard
ids_clients = df["SK_ID_CURR"]

# Supprimer les colonnes non utilisées par le modèle
X = df.drop(columns=["TARGET", "SK_ID_CURR"])

# Charger le modèle
model = joblib.load("../model/best_model_lightgbm.pkl")

# Créer l'explainer
explainer = shap.TreeExplainer(model)

# Calculer les SHAP values
shap_values = explainer.shap_values(X)

# Prendre les valeurs pour la classe 1 (défaut)
if isinstance(shap_values, list):
    shap_values_to_use = shap_values[1]
    base_value = explainer.expected_value[1]
else:
    shap_values_to_use = shap_values
    base_value = explainer.expected_value

# Sauver les SHAP values (par client, local)
df_local=pd.DataFrame(shap_values_to_use)
df_local.to_parquet("shap_values_clients.parquet")

# Sauver les features utilisées pour chaque client
X.to_parquet("X_features_clients.parquet")

# Sauver les IDs clients associés
ids_clients.to_csv("ids_clients.csv", index=False)

# Sauver le base value pour SHAP (valeur attendue)
if np.isscalar(base_value):
    df_base = pd.DataFrame({"base_value": [base_value]})
elif isinstance(base_value, (list, np.ndarray)):
    df_base = pd.DataFrame({"base_value": base_value})
else:
    raise ValueError("Format inattendu pour base_value :", type(base_value))

df_base.to_parquet("shap_base_value.parquet")


print("Fichiers SHAP sauvegardés avec succès.")

# Moyenne absolue des valeurs SHAP par feature (global importance)
shap_global = pd.DataFrame({
    "feature": X.columns,
    "importance": np.abs(shap_values_to_use).mean(axis=0)
}).sort_values(by="importance", ascending=False)

shap_global.to_parquet("shap_global_importance")


Fichiers SHAP sauvegardés avec succès.


C:\Users\alaro\anaconda3\envs\scoring-env\lib\site-packages\shap\explainers\_tree.py:583: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
